In [12]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
dummy_data = pd.read_csv('/Users/henry/Desktop/dummdata - Sheet1.csv')

#Buckets + outlier detection and remove/replacement

#Class for bucketing Data
#Class for outlier detection/replcaement or removal
#Class for one hot encoding
#Class for scaling numerical values

In [170]:
#either min or max bucket has to be inclusive add option. 
class Bucketing_Data:
    
    def __init__(self, raw_data):
        self.raw_data = raw_data
        if isinstance(raw_data, pd.DataFrame):
            pass
        else:
            raise ValueError('Must pass in a dataframe')
    
    #Directs bucketing to the correct funciton. 
    def bucketing(self, bucket_type='even_compartments', columns=[], Number_of_buckets = 10, bucket_array =[], replace = False, append=True,
                 min_bucket_size = None, max_bucket_size = None, bottom_bucket_inclusive =True):
        if bucket_type == 'even_entries':
            self.even_entry_buckets(columns, Number_of_buckets)
        elif bucket_type == 'even_bucket_sizes':
            data_T = self.even_size_buckets(columns, Number_of_buckets,replace,append, min_bucket_size,max_bucket_size, bottom_bucket_inclusive)
            return data_T
        elif bucket_type == 'custom':
            data_T = self.custom_buckets(columns, bucket_array, replace, append, bottom_bucket_inclusive)
            return data_T
        elif bucket_type == 'min_max_bs':
            self.minmax_bs()
        else:
            raise ValueError('invalid bucket type options are: custom, even_entries, even_bucket_sizes and min_max_bs')
            
    #Gives buckets with an around an even number of entries per bucket with given number of buckets.
    def even_entry_buckets(self,columns = []):
        print('even_entries_per_bucket_buckets')
        
    #Gives even bucket sizes
    def even_size_buckets(self,columns, Number_of_buckets,replace,append, min_bucket_size,max_bucket_size, bottom_bucket_inclusive):
        if columns[0]:
            pass
        else:
            raise ValueError('columns must be a list of columns')
        if str(Number_of_buckets).isnumeric():
            pass
        else:
            raise ValueError('Number of buckets must be a number')
        if replace == True or replace == False:
            pass
        else:
            raise ValueError('replace must be boolean')
        if append == True or append == False:
            pass
        else:
            raise ValueError('append must be boolean')
        
        print('even bucket sizes: ', columns)
        return_frame = pd.DataFrame()
        if bottom_bucket_inclusive == True: 
            for item in columns:
                try: 
                    bucketed_data = pd.DataFrame(self.raw_data[item], columns=[item])
                except:
                    raise ValueError('no such column: ', item)
                if str(min_bucket_size).isnumeric():
                    minimum = min_bucket_size
                else:
                    minimum = min(data)
                if str(max_bucket_size).isnumeric():
                    maximum = max_bucket_size
                else:
                    maximum = max(data)
                try:
                    bucket_size = (maximum-minimum)/Number_of_buckets
                    print('\ncolumn: ', item, '\nmin:', str(minimum), ' max:', str(maximum), ' bucket size:', str(bucket_size))
                except:
                    raise ValueError('Bucket column ', item, ' must be numeric')

                bucket_start = minimum
                

                for bucket in range(0, int(Number_of_buckets)):
                    bucket_end = bucket_start + bucket_size
                    if bucket == 0:
                        less_than_min_column_name = str(item) + '_<_' + str(minimum)
                        bucketed_data[less_than_min_column_name] = bucketed_data.apply(lambda x: 1 if x[item] < minimum else 0,axis=1)


                    column_name = str(item)+ '_>=_' + str(bucket_start) + '_<' + str(bucket_end) 
                    bucketed_data[column_name] = bucketed_data.apply(lambda x: 1 if x[item]>= bucket_start and x[item] < bucket_end else 0,axis=1)
                    if bucket == int(Number_of_buckets) -1:
                        more_than_max_column_name = str(item) + '_>=_' + str(maximum)
                        bucketed_data[more_than_max_column_name] = bucketed_data.apply(lambda x: 1 if x[item]>= maximum else 0,axis=1)

                    bucket_start +=bucket_size
                return_frame = pd.concat([return_frame, bucketed_data], axis=1)
        else:
            for item in columns:
                try: 
                    bucketed_data = pd.DataFrame(self.raw_data[item], columns=[item])
                except:
                    raise ValueError('no such column: ', item)
                if str(min_bucket_size).isnumeric():
                    minimum = min_bucket_size
                else:
                    minimum = min(data)
                if str(max_bucket_size).isnumeric():
                    maximum = max_bucket_size
                else:
                    maximum = max(data)
                try:
                    bucket_size = (maximum-minimum)/Number_of_buckets
                    print('\ncolumn: ', item, '\nmin:', str(minimum), ' max:', str(maximum), ' bucket size:', str(bucket_size))
                except:
                    raise ValueError('Bucket column ', item, ' must be numeric')

                bucket_start = minimum
                
                for bucket in range(0, int(Number_of_buckets)):
                    bucket_end = bucket_start + bucket_size
                    if bucket == 0:
                        less_than_min_column_name = str(item) + '_<=_' + str(minimum)
                        bucketed_data[less_than_min_column_name] = bucketed_data.apply(lambda x: 1 if x[item] <= minimum else 0,axis=1)


                    column_name = str(item)+ '_>_' + str(bucket_start) + '_<=' + str(bucket_end) 
                    bucketed_data[column_name] = bucketed_data.apply(lambda x: 1 if x[item] > bucket_start and x[item] <= bucket_end else 0,axis=1)
                    if bucket == int(Number_of_buckets) -1:
                        more_than_max_column_name = str(item) + '_>_' + str(maximum)
                        bucketed_data[more_than_max_column_name] = bucketed_data.apply(lambda x: 1 if x[item]> maximum else 0,axis=1)

                    bucket_start +=bucket_size
                return_frame = pd.concat([return_frame, bucketed_data], axis=1)
                
                
        if append == True and replace==True:
            return_frame = pd.concat([self.raw_data, return_frame],axis=1)
            return_frame = return_frame.drop(columns=columns)
            return return_frame
        elif append==True and replace==False:
            return_frame = return_frame.drop(columns=columns)
            return_frame = pd.concat([self.raw_data, return_frame],axis=1)
            return return_frame
        elif append==False and replace==True:
            return_frame = return_frame.drop(columns=columns)
            return return_frame
        elif append==False and replace==False:
            return return_frame
       
    
    
    
    
    def custom_buckets(self, columns,bucket_array, replace, append, bottom_bucket_inclusive):
        bucket_array.sort()
        try:
            columns[0]
            bucket_array[0]
        except:
            raise ValueError('columns must be a list of columns and bucket_array must be an array of bottom bucket sizes')
        if replace == True or replace == False:
            pass
        else:
            raise ValueError('replace must be boolean')
        if append == True or append == False:
            pass
        else:
            raise ValueError('append must be boolean')
        
        print('custom bucket sizes: ', columns)
        return_frame = pd.DataFrame()
        if bottom_bucket_inclusive == True: 
            for item in columns:
                try: 
                    bucketed_data = pd.DataFrame(self.raw_data[item], columns=[item])
                except:
                    raise ValueError('no such column: ', item)
                
                try:
                   
                    print('\ncolumn: ', item, 'columns:', str(bucket_array))
                    test = float(bucketed_data[item].iloc[0])
                except:
                    raise ValueError('Bucket column ', item, ' must be numeric')
                    
                for bucket in range(0,len(bucket_array)+1):
                    try:
                        if bucket != 0 and bucket != (len(bucket_array)):
                            bucket_start = bucket_array[bucket-1]
                            bucket_end = bucket_array[bucket]
                            new_column = item +'_>='+ str(bucket_start)+ '_<' + str(bucket_end)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] < bucket_end and x[item] >= bucket_start else 0,axis=1)
                        elif bucket == 0 and bucket != len(bucket_array) + 1:
                            bucket_end = bucket_array[bucket]
                            new_column = item + '_<' + str(bucket_end)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] < bucket_end else 0,axis=1)
                        else:
                            bucket_start = bucket_array[bucket-1]
                            new_column = item + '_>=' + str(bucket_start)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] >= bucket_start else 0,axis=1)
                    except:
                        raise ValueError('Buckets must be an array of bottom bucket values')
                return_frame = pd.concat([return_frame, bucketed_data], axis=1)
                
                
            if append == True and replace==True:
                return_frame = pd.concat([self.raw_data, return_frame],axis=1)
                return_frame = return_frame.drop(columns=columns)
                return return_frame
            elif append==True and replace==False:
                return_frame = return_frame.drop(columns=columns)
                return_frame = pd.concat([self.raw_data, return_frame],axis=1)
                return return_frame
            elif append==False and replace==True:
                return_frame = return_frame.drop(columns=columns)
                return return_frame
            elif append==False and replace==False:
                return return_frame
       
        else: 

            for item in columns:
                try: 
                    bucketed_data = pd.DataFrame(self.raw_data[item], columns=[item])
                except:
                    raise ValueError('no such column: ', item)

                try:

                    print('\ncolumn: ', item, 'columns:', str(bucket_array))
                    test = float(bucketed_data[item].iloc[0])
                except:
                    raise ValueError('Bucket column ', item, ' must be numeric')

                for bucket in range(0,len(bucket_array)+1):
                    try:
                        if bucket != 0 and bucket != (len(bucket_array)):
                            bucket_start = bucket_array[bucket-1]
                            bucket_end = bucket_array[bucket]
                            new_column = item +'_>'+ str(bucket_start)+ '_=<' + str(bucket_end)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] <= bucket_end and x[item] > bucket_start else 0,axis=1)
                        elif bucket == 0 and bucket != len(bucket_array) + 1:
                            bucket_end = bucket_array[bucket]
                            new_column = item + '_<=' + str(bucket_end)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] <= bucket_end else 0,axis=1)
                        else:
                            bucket_start = bucket_array[bucket-1]
                            new_column = item + '_>' + str(bucket_start)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] > bucket_start else 0,axis=1)
                    except:
                        raise ValueError('Buckets must be an array of bottom bucket values')
                return_frame = pd.concat([return_frame, bucketed_data], axis=1)


        if append == True and replace==True:
            return_frame = pd.concat([self.raw_data, return_frame],axis=1)
            return_frame = return_frame.drop(columns=columns)
            return return_frame
        elif append==True and replace==False:
            return_frame = return_frame.drop(columns=columns)
            return_frame = pd.concat([self.raw_data, return_frame],axis=1)
            return return_frame
        elif append==False and replace==True:
            return_frame = return_frame.drop(columns=columns)
            return return_frame
        elif append==False and replace==False:
            return return_frame


                    
       
                
            
        
    
        
        
        

In [172]:
#Custom buckets
data = Bucketing_Data(raw_data=dummy_data).bucketing(bucket_type='custom', columns=['temperature', 'windspeed'], bucket_array=[30,40,50,70,35], append=True, replace=False, bottom_bucket_inclusive=False)

display(data)


custom bucket sizes:  ['temperature', 'windspeed']

column:  temperature columns: [30, 35, 40, 50, 70]

column:  windspeed columns: [30, 35, 40, 50, 70]


date         climate  temperature  windspeed  temperature_<=30  \
0     1/1/2020  humid-seasonal           57         27                 0   
1     1/2/2020       humid-hot           59         11                 0   
2     1/3/2020       humid-hot           89         12                 0   
3     1/4/2020  humid-seasonal           37         33                 0   
4     1/5/2020         dry-hot           58         23                 0   
..         ...             ...          ...        ...               ...   
494   5/9/2021       humid-hot           65         18                 0   
495  5/10/2021  humid-seasonal           92         34                 0   
496  5/11/2021         dry-hot           99         32                 0   
497  5/12/2021  humid-seasonal           90         15                 0   
498  5/13/2021       humid-hot           49         16                 0   

     temperature_>30_=<35  temperature_>35_=<40  temperature_>40_=<50  \
0                       0                     0                     0   
1                       0                     0                     0   
2                       0                     0                     0   
3                       0                     1                     0   
4                       0                     0                     0   
..                    ...                   ...                   ...   
494                     0                     0                     0   
495                     0                     0                     0   
496                     0                     0                     0   
497                     0                     0                     0   
498                     0                     0                     1   

     temperature_>50_=<70  temperature_>70  windspeed_<=30  \
0                       1                0               1   
1                       1                0               1   
2                       0                1               1   
3                       0                0               0   
4                       1                0               1   
..                    ...              ...             ...   
494                     1                0               1   
495                     0                1               0   
496                     0                1               0   
497                     0                1               1   
498                     0                0               1   

     windspeed_>30_=<35  windspeed_>35_=<40  windspeed_>40_=<50  \
0                     0                   0                   0   
1                     0                   0                   0   
2                     0                   0                   0   
3                     1                   0                   0   
4                     0                   0                   0   
..                  ...                 ...                 ...   
494                   0                   0                   0   
495                   1                   0                   0   
496                   1                   0                   0   
497                   0                   0                   0   
498                   0                   0                   0   

     windspeed_>50_=<70  windspeed_>70  
0                     0              0  
1                     0              0  
2                     0              0  
3                     0              0  
4                     0              0  
..                  ...            ...  
494                   0              0  
495                   0              0  
496                   0              0  
497                   0              0  
498                   0              0  

[499 rows x 16 columns]

In [143]:
# #Even bucket sizes. 
# my_data = Bucketing_Data(raw_data=dummy_data).bucketing(bucket_type='even_bucket_sizes', columns=['temperature'],Number_of_buckets=5, replace=False,append=True, min_bucket_size=30, max_bucket_size=70, bottom_bucket_inclusive=False)
# my_data.head(50)




even bucket sizes:  ['temperature']

column:  temperature 
min: 30  max: 70  bucket size: 8.0


date         climate  temperature  windspeed  temperature_<=_30  \
0    1/1/2020  humid-seasonal           57         27                  0   
1    1/2/2020       humid-hot           59         11                  0   
2    1/3/2020       humid-hot           89         12                  0   
3    1/4/2020  humid-seasonal           37         33                  0   
4    1/5/2020         dry-hot           58         23                  0   
5    1/6/2020         dry-hot           59         27                  0   
6    1/7/2020       humid-hot           92         37                  0   
7    1/8/2020         dry-hot            9         20                  1   
8    1/9/2020         dry-hot           69         18                  0   
9   1/10/2020    dry-seasonal           30         37                  1   
10  1/11/2020         dry-hot           76         10                  0   
11  1/12/2020       humid-hot           25         14                  1   
12  1/13/2020    dry-seasonal           78         12                  0   
13  1/14/2020  humid-seasonal           82         11                  0   
14  1/15/2020       humid-hot           58         19                  0   
15  1/16/2020         dry-hot           74         39                  0   
16  1/17/2020  humid-seasonal           29         20                  1   
17  1/18/2020       humid-hot           74         29                  0   
18  1/19/2020  humid-seasonal           51         24                  0   
19  1/20/2020       humid-hot           60         33                  0   
20  1/21/2020    dry-seasonal           81         33                  0   
21  1/22/2020  humid-seasonal           88         26                  0   
22  1/23/2020       humid-hot           23         11                  1   
23  1/24/2020       humid-hot          100         21                  0   
24  1/25/2020    dry-seasonal           51         19                  0   
25  1/26/2020  humid-seasonal            8         33                  1   
26  1/27/2020  humid-seasonal           35         11                  0   
27  1/28/2020         dry-hot           86         33                  0   
28  1/29/2020  humid-seasonal           73         12                  0   
29  1/30/2020         dry-hot           18         17                  1   
30  1/31/2020  humid-seasonal           26         37                  1   
31   2/1/2020       humid-hot          100         19                  0   
32   2/2/2020  humid-seasonal           65         20                  0   
33   2/3/2020         dry-hot           74         19                  0   
34   2/4/2020         dry-hot           24         39                  1   
35   2/5/2020       humid-hot           72         39                  0   
36   2/6/2020  humid-seasonal            3         17                  1   
37   2/7/2020       humid-hot           42         36                  0   
38   2/8/2020  humid-seasonal            6         15                  1   
39   2/9/2020       humid-hot           69         25                  0   
40  2/10/2020       humid-hot           89         15                  0   
41  2/11/2020       humid-hot           91         21                  0   
42  2/12/2020    dry-seasonal            8         31                  1   
43  2/13/2020         dry-hot           11         26                  1   
44  2/14/2020    dry-seasonal           24         22                  1   
45  2/15/2020    dry-seasonal           70         39                  0   
46  2/16/2020       humid-hot           62         32                  0   
47  2/17/2020       humid-hot           66         19                  0   
48  2/18/2020       humid-hot           88         40                  0   
49  2/19/2020  humid-seasonal           13         28                  1   

    temperature_>_30_<=38.0  temperature_>_38.0_<=46.0  \
0                         0                          0   
1               

[0, 1, 3, 6]

In [447]:
#Even entries buckets going to have to do two 1 where bottom inclusive and 1 where top is inslusive. 
Num_buckets = 10
column = 'temperature'
trial = pd.DataFrame(dummy_data[column])
trial = trial.sort_values(by=column)
entries_per_bucket = round(max(trial.index)/Num_buckets,0)
entries_per_bucket
start_index = 0
last_start_value = min(trial[column])
last_end_value = min(trial[column])
trial['original_index'] = trial.index
trial = trial.reset_index(drop=True)
for i in range(0, Num_buckets):
    end_index = start_index + entries_per_bucket
    start_value = last_end_value
    try: 
        end_value = trial[trial.index==end_index][column].values[0]
    except:
        end_value = max(trial[column])
    print(end_value)
    
    
    last_end_value = end_value
    start_index = end_index
    


11
19
31
42
52
62
72
81
92
100
